## 캐글 타이타닉 생존률

### 파이토치로 수행

#### 모듈 임포트

In [176]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

#### 데이터셋 로드

- KaggleHub 사용법
- kaggle 사이트에서 설정한 토큰 확인, kaggle.json

In [7]:
!pip install kagglehub


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import kagglehub

In [ ]:
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [90]:
# 종류가 디게 많음. 잘 찾아야 함!!
path = kagglehub.dataset_download("mustafahakan/titanic-traintest-data")

print("Path to dataset files:", path)

100%|██████████| 33.1k/33.1k [00:00<00:00, 1.88MB/s]

Extracting files...
Path to dataset files: C:\Users\perso\.cache\kagglehub\datasets\mustafahakan\titanic-traintest-data\versions\1


In [108]:
df_train = pd.read_csv(rf'C:\Users\perso\.cache\kagglehub\datasets\mustafahakan\titanic-traintest-data\versions\1\train.csv')
df_test = pd.read_csv(rf'C:\Users\perso\.cache\kagglehub\datasets\mustafahakan\titanic-traintest-data\versions\1\test.csv')

In [109]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [110]:
df_train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [111]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [112]:
df_test.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


#### Data Pre-processing

##### 불필요한 데이터 삭제
- 분석에 필요없는 Name, Ticket과 결측치가 너무 많은 Cabin 컬럼 삭제

In [113]:
df_train.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [114]:
df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

##### 컬럼의 object 타입을 numerical 타입으로 변경

- Sex, Embarked 의 값을, get_dummies 함수를 이용해 **label encoding**, one-hot encoding 등 처리
- Male/Female 1/0, C/Q/S 0/1/2 로 변경

In [120]:
le = preprocessing.LabelEncoder()
le.fit(df_train['Sex'])

LabelEncoder()

In [121]:
df_train['Sex'] = le.transform(df_train['Sex'])

In [122]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,S
1,2,1,1,0,38.0,1,0,71.2833,C
2,3,1,3,0,26.0,0,0,7.9250,S
3,4,1,1,0,35.0,1,0,53.1000,S
4,5,0,3,1,35.0,0,0,8.0500,S


In [124]:
df_test['Sex'] = le.transform(df_test['Sex'])

In [126]:
df_test.tail()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
413,1305,3,1,NaN,0,0,8.0500,S
414,1306,1,0,39.0,0,0,108.9000,C
415,1307,3,1,38.5,0,0,7.2500,S
416,1308,3,1,NaN,0,0,8.0500,S
417,1309,3,1,NaN,1,1,22.3583,C


In [127]:
le2 = preprocessing.LabelEncoder()
le2.fit(df_train['Embarked'])

LabelEncoder()

In [128]:
df_train['Embarked'] = le2.transform(df_train['Embarked'])

In [130]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,2
1,2,1,1,0,38.0,1,0,71.2833,0
2,3,1,3,0,26.0,0,0,7.9250,2
3,4,1,1,0,35.0,1,0,53.1000,2
4,5,0,3,1,35.0,0,0,8.0500,2


In [131]:
df_test['Embarked'] = le2.transform(df_test['Embarked'])

In [132]:
df_test.tail()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
413,1305,3,1,NaN,0,0,8.0500,2
414,1306,1,0,39.0,0,0,108.9000,0
415,1307,3,1,38.5,0,0,7.2500,2
416,1308,3,1,NaN,0,0,8.0500,2
417,1309,3,1,NaN,1,1,22.3583,0


##### 결측값 확인

In [137]:
df_train.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [136]:
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())

In [134]:
df_test.isna().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
Embarked        0
dtype: int64

In [138]:
df_test['Age'] = df_test['Age'].fillna(df_test['Age'].mean())

In [139]:
df_test['Fare'] = df_test['Fare'].fillna(df_test['Fare'].mean())

In [142]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,2
1,2,1,1,0,38.0,1,0,71.2833,0
2,3,1,3,0,26.0,0,0,7.9250,2
3,4,1,1,0,35.0,1,0,53.1000,2
4,5,0,3,1,35.0,0,0,8.0500,2


In [143]:
df_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,7.8292,1
1,893,3,0,47.0,1,0,7.0000,2
2,894,2,1,62.0,0,0,9.6875,1
3,895,3,1,27.0,0,0,8.6625,2
4,896,3,0,22.0,1,1,12.2875,2


##### Scaler
- 데이터 고유의 특성과 분포가 존재. 학습에 크게 작용하는 특성 feature과 그렇지 않은 특성이 발생
- StandardScalar를 사용 데이터를 Scaling
    - 기존 변수의 범위를 정규분포로 변환
    - 모든 특성의 평균을 0, 분산을 1로
    

In [145]:
Y_train = df_train['Survived'].values

In [147]:
test_PassengerId = df_test['PassengerId'].values

In [148]:
Scaler1 = StandardScaler()
Scaler2 = StandardScaler()

In [149]:
train_columns = df_train.columns
train_columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked'],
      dtype='object')

In [150]:
test_columns = df_test.columns
test_columns

Index(['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [151]:
df_train = pd.DataFrame(Scaler1.fit_transform(df_train))
df_test = pd.DataFrame(Scaler2.fit_transform(df_test))

In [152]:
df_train.head()

,0,1,2,3,4,5,6,7,8
0,-1.730108,-0.789272,0.827377,0.737695,-0.592481,0.432793,-0.473674,-0.502445,0.581114
1,-1.726220,1.266990,-1.566107,-1.355574,0.638789,0.432793,-0.473674,0.786845,-1.938460
2,-1.722332,1.266990,0.827377,-1.355574,-0.284663,-0.474545,-0.473674,-0.488854,0.581114
3,-1.718444,1.266990,-1.566107,-1.355574,0.407926,0.432793,-0.473674,0.420730,0.581114
4,-1.714556,-0.789272,0.827377,0.737695,0.407926,-0.474545,-0.473674,-0.486337,0.581114


In [153]:
df_train.columns = train_columns
df_test.columns = test_columns

In [154]:
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,-1.730108,-0.789272,0.827377,0.737695,-0.592481,0.432793,-0.473674,-0.502445,0.581114
1,-1.726220,1.266990,-1.566107,-1.355574,0.638789,0.432793,-0.473674,0.786845,-1.938460
2,-1.722332,1.266990,0.827377,-1.355574,-0.284663,-0.474545,-0.473674,-0.488854,0.581114
3,-1.718444,1.266990,-1.566107,-1.355574,0.407926,0.432793,-0.473674,0.420730,0.581114
4,-1.714556,-0.789272,0.827377,0.737695,0.407926,-0.474545,-0.473674,-0.486337,0.581114


In [158]:
X_train = df_train.iloc[:, 2:].values

In [160]:
X_train.shape

(891, 7)

#### 파이토치 신경망

##### 모델생성

In [161]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(7, 448),
            nn.ReLU(),
            nn.Dropout(0.2))
        self.layer2 = nn.Sequential(
            nn.Linear(448, 448),
            nn.ReLU(),
            nn.Dropout(0.2))
        self.layer3 = nn.Linear(448, 2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [162]:
model = Network()
model

Network(
  (layer1): Sequential(
    (0): Linear(in_features=7, out_features=448, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
  )
  (layer2): Sequential(
    (0): Linear(in_features=448, out_features=448, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
  )
  (layer3): Linear(in_features=448, out_features=2, bias=True)
)

##### 데이터셋, 데이터로더

In [164]:
X_train.shape

(891, 7)

In [166]:
Y_train.shape

(891,)

In [165]:
torch.FloatTensor(X_train)

tensor([[ 0.8274,  0.7377, -0.5925,  ..., -0.4737, -0.5024,  0.5811],
        [-1.5661, -1.3556,  0.6388,  ..., -0.4737,  0.7868, -1.9385],
        [ 0.8274, -1.3556, -0.2847,  ..., -0.4737, -0.4889,  0.5811],
        ...,
        [ 0.8274, -1.3556,  0.0000,  ...,  2.0089, -0.1763,  0.5811],
        [-1.5661,  0.7377, -0.2847,  ..., -0.4737, -0.0444, -1.9385],
        [ 0.8274,  0.7377,  0.1771,  ..., -0.4737, -0.4924, -0.6787]])

In [167]:
dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(Y_train))

In [174]:
dataset.tensors

(tensor([[ 0.8274,  0.7377, -0.5925,  ..., -0.4737, -0.5024,  0.5811],
         [-1.5661, -1.3556,  0.6388,  ..., -0.4737,  0.7868, -1.9385],
         [ 0.8274, -1.3556, -0.2847,  ..., -0.4737, -0.4889,  0.5811],
         ...,
         [ 0.8274, -1.3556,  0.0000,  ...,  2.0089, -0.1763,  0.5811],
         [-1.5661,  0.7377, -0.2847,  ..., -0.4737, -0.0444, -1.9385],
         [ 0.8274,  0.7377,  0.1771,  ..., -0.4737, -0.4924, -0.6787]]),
 tensor([0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
         0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
         1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,

In [175]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

##### 옵티마이저 등

In [193]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

#### 훈련

In [189]:
from tqdm.autonotebook import tqdm # 프로그래스바 시각화 모듈

In [194]:
nb_epochs = 1000
train_loss_min = np.Inf

for epoch in tqdm( range(nb_epochs)):
    num_right = 0
    train_loss = 0

    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples

        prediction = model(x_train)
        loss = criterion(prediction, y_train.long())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        labels = torch.argmax(prediction, dim=1)
        num_right += torch.sum(labels == y_train)
        train_loss += loss.item() * len(x_train)

    train_loss = train_loss / len(X_train)
    if (epoch + 1) % 20 == 0:
        print(f'Epoch {epoch+1}/{nb_epochs}, Prediction : {num_right}/{len(X_train)}, Cost : {train_loss}')
    if train_loss <= train_loss_min:
        print(F'>>>>> Loss decreased ({train_loss_min:6f} ===> {train_loss:6f}). Saving the model!')
        torch.save(model.state_dict(), './data/model.pt')
        train_loss_min = train_loss
        
print('Training Ended!')

  0%|          | 0/1000 [00:00<?, ?it/s]

>>>>> Loss decreased (   inf ===> 0.225784). Saving the model!
>>>>> Loss decreased (0.225784 ===> 0.211321). Saving the model!
>>>>> Loss decreased (0.211321 ===> 0.207865). Saving the model!
>>>>> Loss decreased (0.207865 ===> 0.205815). Saving the model!
Epoch 20/1000, Prediction : 802/891, Cost : 0.21132579228425802
>>>>> Loss decreased (0.205815 ===> 0.202076). Saving the model!
Epoch 40/1000, Prediction : 805/891, Cost : 0.20470662739474899
Epoch 60/1000, Prediction : 805/891, Cost : 0.21552160256840164
>>>>> Loss decreased (0.202076 ===> 0.201514). Saving the model!
Epoch 80/1000, Prediction : 803/891, Cost : 0.21715527773739646
>>>>> Loss decreased (0.201514 ===> 0.198465). Saving the model!
Epoch 100/1000, Prediction : 805/891, Cost : 0.23058505361329024
Epoch 120/1000, Prediction : 802/891, Cost : 0.22413166201810378
Epoch 140/1000, Prediction : 807/891, Cost : 0.21389905735432233
Epoch 160/1000, Prediction : 805/891, Cost : 0.2094717575728425
>>>>> Loss decreased (0.198465 =

#### 예측

In [197]:
df_test.iloc[:,1:].values

array([[ 0.87348191,  0.75592895,  0.3349926 , ..., -0.4002477 ,
        -0.49840706, -0.47091535],
       [ 0.87348191, -1.32287566,  1.32553003, ..., -0.4002477 ,
        -0.51327429,  0.70076689],
       [-0.31581919,  0.75592895,  2.51417495, ..., -0.4002477 ,
        -0.46508846, -0.47091535],
       ...,
       [ 0.87348191,  0.75592895,  0.65196458, ..., -0.4002477 ,
        -0.50879189,  0.70076689],
       [ 0.87348191,  0.75592895,  0.        , ..., -0.4002477 ,
        -0.4944482 ,  0.70076689],
       [ 0.87348191,  0.75592895,  0.        , ...,  0.61989583,
        -0.23790598, -1.64259759]])

In [198]:
X_test = torch.FloatTensor(df_test.iloc[:,1:].values)

In [199]:
with torch.no_grad():
    result = model(X_test)

labels = torch.argmax(result, dim=1)
survived = labels.numpy()

In [200]:
survived

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

#### 종료

In [202]:
submission = pd.DataFrame({'PassengerId': test_PassengerId, 'Survived': survived})

In [204]:
submission.tail()

,PassengerId,Survived
413,1305,0
414,1306,1
415,1307,0
416,1308,0
417,1309,1


In [205]:
submission.to_csv('./data/submission.csv', encoding='utf-8', index=False)